In [1]:
#Read in Redlist

import pandas as pd
import py_entitymatching as em
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
style.use('ggplot')

# Read in csv as dataframe
rl = em.read_csv_metadata("tenFeaturesRedlist.csv", encoding="ISO-8859-1", key='id')
# glance at first few rows
rl.head()


Metadata file is not present in the given path; proceeding to read the csv file.


,id,name,genus,family,ecology,countries,threat_paragraph,conservation_paragraph,pop_trend,status,country_count
0,0,Ochlockonee Moccasinshell,Medionidus,Unionidae,Freshwater,"United States (Florida, Georgia);",NaN,NaN,NaN,\n Critically Endangered\n\n\n A1ce\n\n,1
1,1,Nelson's Spiny Pocket Mouse,Heteromys,Heteromyidae,Terrestrial,Guatemala; Mexico (Chiapas);,\n The main threat to this species is the significant forest loss in its habitat. Habitat fra...,"['\n This species occurs in a newly-named national park, Tacana, in Mexico.\n\n \n ']",Decreasing,"\n Endangered\n\n\n B1ab(i,ii,iii,v)\n\n",2
2,2,Comoro Friar,Amauris,Nymphalidae,NaN,Comoros;,NaN,NaN,NaN,"\n Endangered\n\n\n B1+2c, C2b\n\n",1
3,3,"Atlantic Halibut, Halibut",Hippoglossus,Pleuronectidae,NaN,<div>Atlantic  northeast; Atlantic  northwest</div>; Canada (Newfoundland I); Denmark; Far...,NaN,NaN,NaN,\n Endangered\n\n\n A1d\n\n,15
4,10,Oktibbeha Riverlet Crayfish,Hobbseus,Cambaridae,Freshwater,United States (Mississippi);,\n This species is threatened by habitat degradation through residential and urban developmen...,"['\n', <span style=""font-style: italic;""></span>, 'This species has been given the Global Herita...",Unknown,\n Endangered\n\n\n B1ab(iii)\n\n,1


In [2]:
# CME
em.get_key(rl) # get key

# Note: The id column of the  redlist csv doesn't actually look like the object created here (rl).  In the csv,
# id numbers jump around since they represent original row number of the original scraped data csv.  Apparently
# this library can detect what we want and enumerate things in order if you give it a key feature like this.


'id'

In [3]:
#Read in ARKIVE
# Read in csv as dataframe
ar = em.read_csv_metadata("arkive table analysis/finalOutput.csv", encoding="ISO-8859-1")

# glance at first few rows
ar.head()

Metadata file is not present in the given path; proceeding to read the csv file.


,id,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount
0,0,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby (Papasula abbotti),Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula (1),Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0
1,1,Abbotí¢ó»s duiker,Cephalophus spadix,Abbottí¢ó»s duiker (Cephalophus spadix),Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus (1),50 í¢óñ 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1
2,2,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink (Tiliqua adelaidensis),Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua (1),Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1
3,3,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon (Salmo obtusirostris),Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo (1),80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0
4,4,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin (Spheniscus demersus),Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus (1),Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2


In [4]:
# Remove parentheticals from arkive common names
# author: CME
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['common_name'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'common_name'] = noparens
    else:
        error = error + 1

print(error)


0


In [5]:
# Concatenate nickname and common_name fields
# author: @andrewedstrom

# remove parenthesis contents
nameColumn = []
for index, row in ar.iterrows():
    nickname = str(row['scientific_name']) #look here if errors start happening
    common = str(row['common_name'])
    nameColumn.append(nickname + "; " + common)
ar['name'] = nameColumn
ar.head()

,id,scientific_name,nickname,common_name,kingdom,phylum,class,order,family,genus,size,threats,conservation,threats_keywords,conservation_keywords,tCount,cCount,name
0,0,Alcatraz de Abbott; Piquero de Abbott,Papasula abbotti,Abbott's booby,Animalia,Chordata,Aves,Pelecaniformes,Sulidae,Papasula (1),Length: 79 cm (2),Phosphate mining on Christmas Island in the 1960s and 70s caused the destruction of a third of A...,The majority of Abbott's booby nest sites occur within the Christmas Island National Park which ...,,,0,0,Alcatraz de Abbott; Piquero de Abbott; Abbott's booby
1,1,Abbotí¢ó»s duiker,Cephalophus spadix,Abbottí¢ó»s duiker,Animalia,Chordata,Mammalia,Cetartiodactyla,Bovidae,Cephalophus (1),50 í¢óñ 60 kg (2),Like many other duikers Abbott\xe2\x80\x99s duiker is being impacted by a combination of hunting...,Although Abbott\xe2\x80\x99s duiker occurs within several protected areas such as the Kilimanjar...,hunting;,protected;,1,1,Abbotí¢ó»s duiker; Abbottí¢ó»s duiker
2,2,Adelaide pigmy blue-tongue skink; pygmy bluetongue,Tiliqua adelaidensis,Adelaide pygmy bluetongue skink,Animalia,Chordata,Reptilia,Squamata,Scincidae,Tiliqua (1),Male snout-vent length: 3.8 - 10.6 cm (2)Female snout-vent length: 8.8 - 10.7 cm (2)Tail length:...,The Adelaide pygmy bluetongue skink was once feared to be the first of Australia\xe2\x80\x99s ap...,The South Australia Department of Environment and Conservation has produced a National Recovery ...,,protected;,0,1,Adelaide pigmy blue-tongue skink; pygmy bluetongue; Adelaide pygmy bluetongue skink
3,3,Adriatic trout; softmouth trout,Salmo obtusirostris,Adriatic salmon,Animalia,Chordata,Actinopterygii,Salmoniformes,Salmonidae,Salmo (1),80 - 425 g (4),The Adriatic salmon faces a number of threats including overfishing for sport and for food as we...,There are currently no known conservation measures in place for the Adriatic salmon (1) (5). It ...,fishing;overfishing;invasive;pet;,,4,0,Adriatic trout; softmouth trout; Adriatic salmon
4,4,black-footed penguin; jackass penguin,Spheniscus demersus,African penguin,Animalia,Chordata,Aves,Sphenisciformes,Spheniscidae,Spheniscus (1),Length: 60 - 70 cm (2),The population of the African penguin has declined and it is estimated that its current size is ...,The African penguin is protected by its listing on Appendix II of the Convention on Internationa...,fishing;pet;pollution;,cites;protected;,3,2,black-footed penguin; jackass penguin; African penguin


In [6]:
ar.to_csv('finalOutput.csv')

In [7]:
# Remove parentheticals from arkive genus
# author: @andrewedstrom
# Next, actually remove scientific name and replace with common name(s)
#ar = arBeforeRemovingNames # in case script now run completely in order
#ar = ar[ar.animalName.str.contains("\(") == True] # update df to contain only tuples that contain common/nick names
error = 0 # make sure there's no wonky cases

# remove parenthesis contents
for index, row in ar.iterrows():
    genus = str(row['genus'])

    pstart = genus.find("(")
    if pstart !=-1:        
        # extract full common/nick name
        noparens = genus[0:pstart]
        ar.loc[index, 'genus'] = noparens
    else:
        error = error + 1

print(error)

1


In [8]:
# Rule-based blocking over input tables
# first get features that can be used
feature_table = em.get_features_for_blocking(rl, ar)

# CME:  Hmm.. how exactly are these features populated?  What is a 
# left attribute vs a right attribute?

In [9]:
len(feature_table)
feature_table

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated
0,id_id_exm,id,id,None,None,exact_match,<function id_id_exm at 0x109cf7510>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
1,id_id_anm,id,id,None,None,abs_norm,<function id_id_anm at 0x109cf7ae8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
2,id_id_lev_dist,id,id,None,None,lev_dist,<function id_id_lev_dist at 0x109cf7b70>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
3,id_id_lev_sim,id,id,None,None,lev_sim,<function id_id_lev_sim at 0x109cf7400>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
4,family_family_jac_qgm_3_qgm_3,family,family,qgm_3,qgm_3,jaccard,<function family_family_jac_qgm_3_qgm_3 at 0x109cdabf8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
5,family_family_cos_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,cosine,<function family_family_cos_dlm_dc0_dlm_dc0 at 0x109cdaa60>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
6,family_family_jac_dlm_dc0_dlm_dc0,family,family,dlm_dc0,dlm_dc0,jaccard,<function family_family_jac_dlm_dc0_dlm_dc0 at 0x109cda950>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
7,family_family_mel,family,family,None,None,monge_elkan,<function family_family_mel at 0x109cda8c8>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
8,family_family_lev_dist,family,family,None,None,lev_dist,<function family_family_lev_dist at 0x109cda840>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True
9,family_family_lev_sim,family,family,None,None,lev_sim,<function family_family_lev_sim at 0x109cda378>,from py_entitymatching.feature.simfunctions import *\nfrom py_entitymatching.feature.tokenizers ...,True


In [10]:
#list name related features
feature_table[feature_table.left_attribute=='name']

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source,is_auto_generated


In [11]:
# CME -- I'm realizing now that we may want to start with the attribute equivalence
# blocker for names and see where that leaves us.  On my TO-DO.
equivB = em.AttrEquivalenceBlocker()
C0 = equivB.block_tables(rl, ar, 'name', 'name',
                    l_output_attrs=['name', 'genus'], 
                    r_output_attrs=['name', 'genus'])
C0

# lolwat this reveals zero matches?


Requested metadata ( key ) for the given DataFrame is not present in the catalog


KeyError: 'Requested metadata ( key ) for the given DataFrame is not present in the catalog'

In [ ]:
#Do overlap blocking on name to get candidate set
#Perform some kind of blocking to find candidate pairs
ob = em.OverlapBlocker()
C0 = ob.block_tables(rl, ar,'name', 'name', 
                     l_output_attrs=['name', 'genus'], 
                     r_output_attrs=['name', 'genus'])
C0

In [ ]:
#Only keep candidate pairs which share genus
ab = em.AttrEquivalenceBlocker()
C1 = ob.block_candset(C0, 'genus', 'genus')
C1

In [ ]:
#For shits and giggles: 